In [95]:
import os
import numpy as np
import pandas as pd
import psutil
import logging
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Input, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import subprocess
import time
import platform
import asyncio
import nest_asyncio
from collections import deque
from logging.handlers import RotatingFileHandler
import matplotlib.pyplot as plt
import keras_tuner as kt

# 应用 nest_asyncio 以允许嵌套事件循环
nest_asyncio.apply()

# 设置日志并启用日志轮转
log_handler = RotatingFileHandler('system_monitor.log', maxBytes=5000000, backupCount=5)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[log_handler])

# 数据队列大小设置
MAX_METRICS_LEN = 600  # 10分钟的数据

# 警示阈值
ALERT_THRESHOLD = 20

# 收集系统性能数据
def collect_system_metrics(num_samples=10, interval=1):
    metrics_list = []
    for _ in range(num_samples):
        cpu_usage = psutil.cpu_percent(interval=interval)
        memory_info = psutil.virtual_memory()
        memory_usage = memory_info.percent
        disk_io = psutil.disk_io_counters()
        network_io = psutil.net_io_counters()
        metrics_list.append({
            'timestamp': pd.Timestamp.now(),
            'cpu_usage': cpu_usage,
            'memory_usage': memory_usage,
            'disk_read': disk_io.read_bytes,
            'disk_write': disk_io.write_bytes,
            'network_sent': network_io.bytes_sent,
            'network_recv': network_io.bytes_recv
        })
    return metrics_list

# 异常检测
def detect_anomalies(data):
    if len(data) < 50:  # 确保样本数足够
        logging.info("Not enough data to perform anomaly detection.")
        return pd.DataFrame()

    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data.drop(columns=['timestamp']))
    
    model = IsolationForest(contamination=0.01)  # 改进异常检测模型
    scores = model.fit_predict(data_scaled)
    anomaly_scores = (scores < 0).astype(int)
    
    anomalies = data[anomaly_scores > 0]
    return anomalies

# 防止重复修复机制
last_remediation_time = 0
remediation_cooldown = 5  # 冷却时间

def execute_remediation_script(issue_type):
    global last_remediation_time
    current_time = time.time()
    if current_time - last_remediation_time < remediation_cooldown:
        logging.info(f"Skipping remediation due to cooldown: {remediation_cooldown} seconds.")
        return
    
    try:
        logging.info(f"Executing remediation script for issue: {issue_type}...")
        if issue_type == "high_cpu":
            subprocess.run(["echo", "Simulating restart of critical_service"], check=True)
        elif issue_type == "memory_leak":
            subprocess.run(["echo", "Simulating killing of memory_hog_process"], check=True)
        if platform.system() == 'Darwin':
            subprocess.run(["osascript", "-e", f'display notification \"Issue {issue_type} detected and remediated\" with title \"AIOps Alert\"'])
        logging.info("Remediation script executed successfully.")
        last_remediation_time = current_time  # 更新修复时间
    except subprocess.CalledProcessError as e:
        logging.error(f"Error occurred while executing remediation script: {e}")

# 创建BiLSTM模型以用于调参
def build_bilstm_model(hp):
    model = Sequential()

    model.add(Input(shape=(50, 6)))

    # 调节LSTM单元数
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Bidirectional(LSTM(units=hp.Int(f'units_{i}', min_value=50, max_value=400, step=50),
                                     activation='relu', return_sequences=True if i < 2 else False,
                                     kernel_regularizer=tf.keras.regularizers.l2(0.001))))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1))

    model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='mse')

    return model

# 调参和训练
def tune_and_train_model(X_train, y_train):
    tuner = kt.Hyperband(build_bilstm_model,
                         objective='val_loss',
                         max_epochs=50,
                         factor=3,
                         directory='tuner_logs',
                         project_name='bilstm_tuning')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])

    best_model = tuner.get_best_models(num_models=1)[0]

    return best_model

# 数据预处理，构建BiLSTM模型，并进行预测
def prepare_bilstm_model_and_predict(train_data, test_data, n_steps=50):
    scaler = MinMaxScaler(feature_range=(0, 1))

    # 训练数据处理
    train_data_scaled = train_data.copy()
    features_to_scale = ['cpu_usage', 'memory_usage', 'disk_read', 'disk_write', 'network_sent', 'network_recv']
    train_data_scaled[features_to_scale] = scaler.fit_transform(train_data[features_to_scale])
    
    X_train, y_train = [], []
    for i in range(n_steps, len(train_data_scaled)):
        X_train.append(train_data_scaled.iloc[i-n_steps:i].drop(columns=['timestamp']).values)
        y_train.append(train_data_scaled['cpu_usage'].iloc[i])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))

    # 使用自动调参后的模型
    model = tune_and_train_model(X_train, y_train)

    # 测试数据处理
    test_data_scaled = test_data.copy()
    test_data_scaled[features_to_scale] = scaler.transform(test_data[features_to_scale])
    
    X_test = []
    for i in range(n_steps, len(test_data_scaled)):
        X_test.append(test_data_scaled.iloc[i-n_steps:i].drop(columns=['timestamp']).values)
    X_test = np.array(X_test)
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    # 预测
    predictions = model.predict(X_test)

    # 检查 predictions 是否是二维的
    if predictions.ndim == 3:
        predictions = predictions[:, :, 0]  # 如果是三维的，选择第三维度中的第一个元素

    # `predictions` 现在应该是二维数组，形状为 (样本数, 1)
    # 重新调整 `padding_shape` 的定义
    padding_shape = (predictions.shape[0], len(features_to_scale) - 1)

    # 确保填充的 zeros 数组和 predictions 有相同的维度
    predictions_with_padding = np.concatenate([predictions, np.zeros(padding_shape)], axis=1)

    # 对连接后的数组进行逆缩放
    predictions = scaler.inverse_transform(predictions_with_padding)[:, 0]

    # 检查预测值是否超过警示阈值
    high_usage = predictions > ALERT_THRESHOLD
    if np.any(high_usage):
        logging.warning("High CPU usage predicted!")
        execute_remediation_script("potential_high_cpu")

    # 将预测值与时间戳进行合并
    forecast = pd.DataFrame({
        'timestamp': test_data['timestamp'].iloc[n_steps:].values,
        'predicted_cpu_usage': predictions.flatten()
    })

    return forecast


# 实时监控系统性能
async def monitor_system():
    try:
        logging.info("Starting system monitoring...")
        start_time = time.time()
        max_duration = 600  # 10 分钟

        metrics_list = deque(maxlen=MAX_METRICS_LEN)  # 使用双端队列限制数据大小

        # 初始化空的DataFrame用于记录结果
        results_df = pd.DataFrame()

        while time.time() - start_time < max_duration:
            metrics_batch = collect_system_metrics(num_samples=10)
            metrics_list.extend(metrics_batch)
            await asyncio.sleep(1)

        logging.info("Data collection finished. Beginning analysis...")

        data = pd.DataFrame(metrics_list)

        # 使用前80%的数据进行训练，后20%进行预测
        train_size = int(len(data) * 0.8)
        train_data = data.iloc[:train_size]
        test_data = data.iloc[train_size:]

        # 模型训练与预测
        forecast = prepare_bilstm_model_and_predict(train_data, test_data)

        # 合并真实数据和预测数据
        merged_data = pd.merge_asof(test_data[['timestamp', 'cpu_usage']], forecast, on='timestamp')

        # 绘制图表
        plt.figure(figsize=(12, 6))
        plt.plot(merged_data['timestamp'], merged_data['cpu_usage'], label='实际', color='blue')
        plt.plot(merged_data['timestamp'], merged_data['predicted_cpu_usage'], label='预测', color='red', linestyle='--')
        plt.axhline(y=ALERT_THRESHOLD, color='green', linestyle=':', label='警示阈值')
        plt.xlabel('时间')
        plt.ylabel('CPU 使用率')
        plt.title('CPU 使用率 - 实际 vs 预测（最后2分钟）')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    except KeyboardInterrupt:
        logging.info("Monitoring interrupted by user.")
    finally:
        logging.info("Exiting monitoring loop.")

# 启动监控系统
if __name__ == "__main__":
    try:
        asyncio.run(monitor_system())
    except KeyboardInterrupt:
        logging.info("Program terminated by user.")


2024-09-03 20:21:46,960 - INFO - Starting system monitoring...
2024-09-03 20:27:09,964 - INFO - Monitoring interrupted by user.
2024-09-03 20:27:09,968 - INFO - Exiting monitoring loop.


In [100]:
import os
import numpy as np
import pandas as pd
import psutil
import logging
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Input, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import subprocess
import time
import platform
import asyncio
import nest_asyncio
from collections import deque
from logging.handlers import RotatingFileHandler
import matplotlib.pyplot as plt

# 应用 nest_asyncio 以允许嵌套事件循环
nest_asyncio.apply()

# 设置日志并启用日志轮转
log_handler = RotatingFileHandler('system_monitor.log', maxBytes=5000000, backupCount=5)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[log_handler])

# 数据队列大小设置
MAX_METRICS_LEN = 600  # 10分钟的数据
ALERT_THRESHOLD = 20  # 警示阈值

# 初始化数据队列
metrics_list = deque(maxlen=MAX_METRICS_LEN)  # 使用双端队列限制数据大小
start_time = time.time()
max_duration = 600  # 10 分钟

# 监控和收集数据
while time.time() - start_time < max_duration:
    metrics_batch = []
    for _ in range(10):  # 每次收集10个样本
        cpu_usage = psutil.cpu_percent(interval=1)
        memory_info = psutil.virtual_memory()
        memory_usage = memory_info.percent
        disk_io = psutil.disk_io_counters()
        network_io = psutil.net_io_counters()
        metrics_batch.append({
            'timestamp': pd.Timestamp.now(),
            'cpu_usage': cpu_usage,
            'memory_usage': memory_usage,
            'disk_read': disk_io.read_bytes,
            'disk_write': disk_io.write_bytes,
            'network_sent': network_io.bytes_sent,
            'network_recv': network_io.bytes_recv
        })
    metrics_list.extend(metrics_batch)
    await asyncio.sleep(1)

logging.info("Data collection finished. Beginning analysis...")

# 将收集到的数据转换为DataFrame
data = pd.DataFrame(metrics_list)

# 使用前80%的数据进行训练，后20%进行预测
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# 数据预处理与模型训练
scaler = MinMaxScaler(feature_range=(0, 1))

# 训练数据处理
train_data['cpu_usage'] = scaler.fit_transform(train_data[['cpu_usage']])
X_train, y_train = [], []
n_steps = 50
for i in range(n_steps, len(train_data)):
    X_train.append(train_data['cpu_usage'].iloc[i-n_steps:i].values)
    y_train.append(train_data['cpu_usage'].iloc[i])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# 构建BiLSTM模型
model = Sequential([
    Input(shape=(n_steps, 1)),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(400, activation='relu', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001))),
    BatchNormalization(),
    Dropout(0.3),
    Bidirectional(LSTM(200, activation='relu', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001))),
    BatchNormalization(),
    Dropout(0.3),
    Bidirectional(LSTM(100, activation='relu', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001))),
    BatchNormalization(),
    Dropout(0.3),
    Bidirectional(LSTM(50, activation='relu')),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.0001), loss='mse')

# 训练模型
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

# 测试数据处理
test_data['cpu_usage'] = scaler.transform(test_data[['cpu_usage']])
X_test = []
for i in range(n_steps, len(test_data)):
    X_test.append(test_data['cpu_usage'].iloc[i-n_steps:i].values)
X_test = np.array(X_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# 预测
predictions = model.predict(X_test)

# 确保 predictions 是二维的
if predictions.ndim == 3:
    predictions = predictions[:, :, 0]  # 如果是三维的，选择第三维度中的第一个元素

# 将预测值与其他特征合并后进行逆缩放
padding_shape = (predictions.shape[0], 1)
predictions_with_padding = np.concatenate([predictions, np.zeros(padding_shape)], axis=1)
predictions = scaler.inverse_transform(predictions_with_padding)[:, 0]

# 检查预测值是否超过警示阈值
high_usage = predictions > ALERT_THRESHOLD
if np.any(high_usage):
    logging.warning("High CPU usage predicted!")
    subprocess.run(["echo", "Simulating restart of critical_service"], check=True)

# 合并预测值和实际值
forecast = pd.DataFrame({'timestamp': test_data['timestamp'].iloc[n_steps:].values,
                         'predicted_cpu_usage': predictions.flatten()})

# 绘制图表
merged_data = pd.merge_asof(test_data[['timestamp', 'cpu_usage']], forecast, on='timestamp')
plt.figure(figsize=(12, 6))
plt.plot(merged_data['timestamp'], merged_data['cpu_usage'], label='实际', color='blue')
plt.plot(merged_data['timestamp'], merged_data['predicted_cpu_usage'], label='预测', color='red', linestyle='--')
plt.axhline(y=ALERT_THRESHOLD, color='green', linestyle=':', label='警示阈值')
plt.xlabel('时间')
plt.ylabel('CPU 使用率')
plt.title('CPU 使用率 - 实际 vs 预测（最后2分钟）')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


CancelledError: 

In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Input, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow.keras.regularizers import l2

# 生成模拟的时间序列数据
np.random.seed(42)
timestamps = pd.date_range(start="2024-01-01", periods=10000, freq="S")
cpu_usage = np.sin(np.linspace(0, 50, 10000)) + np.random.normal(0, 0.1, 10000) + 0.5

data = pd.DataFrame({"timestamp": timestamps, "cpu_usage": cpu_usage})

# 分割数据为训练集和测试集
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size].copy()
test_data = data.iloc[train_size:].copy()

# 删除时间戳列，确保只使用数值数据
train_data = train_data[['cpu_usage']]

# 数据预处理
scaler = MinMaxScaler(feature_range=(0, 1))
train_data['cpu_usage'] = scaler.fit_transform(train_data[['cpu_usage']])

# 定义模型构建函数
def build_model(hp):
    # 动态调整n_steps
    n_steps = hp.Int('n_steps', min_value=200, max_value=400, step=100)
    
    # 根据n_steps生成训练数据
    X_train, y_train = [], []
    for i in range(n_steps, len(train_data)):
        X_train.append(train_data['cpu_usage'].iloc[i-n_steps:i].values)
        y_train.append(train_data['cpu_usage'].iloc[i])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

    # 构建模型
    model = Sequential()
    model.add(Input(shape=(n_steps, 1)))
    model.add(Conv1D(filters=hp.Int('conv_filters', min_value=32, max_value=128, step=32),
                     kernel_size=3, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(MaxPooling1D(pool_size=2))
    
    for i in range(hp.Int('lstm_layers', 1, 6)):
        model.add(Bidirectional(LSTM(units=hp.Int(f'lstm_units_{i}', min_value=100, max_value=400, step=100),
                                     activation='relu', return_sequences=(i != hp.Int('lstm_layers', 1, 6) - 1),
                                     kernel_regularizer=l2(0.001))))
        model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float(f'dropout_rate_{i}', min_value=0.1, max_value=0.3, step=0.1)))
    
    model.add(Dense(1))

    # 增加学习率调度策略
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='mse')
    
    return model

# 使用Keras Tuner进行超参数搜索
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=15,  # 增加搜索的trial数量
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='cpu_usage_prediction_v2'
)

# 使用EarlyStopping回调
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# 生成训练数据
X_train, y_train = [], []
n_steps = 200  # 设置初始的n_steps
for i in range(n_steps, len(train_data)):
    X_train.append(train_data['cpu_usage'].iloc[i-n_steps:i].values)
    y_train.append(train_data['cpu_usage'].iloc[i])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# 模型搜索与训练
tuner.search_space_summary()
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# 获取最佳模型
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
n_steps = best_hyperparameters.get('n_steps')

# 准备测试数据
test_data['cpu_usage'] = scaler.transform(test_data[['cpu_usage']])
X_test, y_test = [], []
for i in range(n_steps, len(test_data)):
    X_test.append(test_data['cpu_usage'].iloc[i-n_steps:i].values)
    y_test.append(test_data['cpu_usage'].iloc[i])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# 预测
predictions = best_model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# 确保 `predictions` 和 `y_test` 的形状一致
predictions = predictions.flatten()
y_test = y_test[:len(predictions)]  # 确保测试数据长度匹配

# 确保时间戳和预测值对齐
test_timestamps = test_data['timestamp'].iloc[n_steps:n_steps+len(predictions)].values

# 绘制实际值与预测值的图表
plt.figure(figsize=(14, 7))
plt.plot(test_timestamps, y_test, label='实际值', color='blue')
plt.plot(test_timestamps, predictions, label='预测值', color='red', linestyle='--')
plt.xlabel('时间')
plt.ylabel('CPU 使用率')
plt.title('CPU 使用率 - 实际值 vs 预测值')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Trial 2 Complete [00h 13m 43s]
val_loss: nan

Best val_loss So Far: nan
Total elapsed time: 06h 34m 18s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
200               |200               |n_steps
128               |32                |conv_filters
4                 |2                 |lstm_layers
400               |300               |lstm_units_0
0.2               |0.1               |dropout_rate_0
0.0001            |0.01              |learning_rate
300               |100               |lstm_units_1
0.1               |0.1               |dropout_rate_1
400               |None              |lstm_units_2
0.1               |None              |dropout_rate_2

Epoch 1/50
195/195 ━━━━━━━━━━━━━━━━━━━━ 263s 1s/step - loss: nan - val_loss: nan - learning_rate: 1.0000e-04
Epoch 2/50
195/195 ━━━━━━━━━━━━━━━━━━━━ 277s 1s/step - loss: nan - val_loss: nan - learning_rate: 1.0000e-04
Epoch 3/50
195/195 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - loss: nan - val_loss: nan - lea

RuntimeError: Number of consecutive failures exceeded the limit of 3.
